In [15]:
import pyspark
from pyspark import SparkContext
import imageio
import os
import numpy as np
from scipy.ndimage import median_filter

In [8]:
def readImg(path):
    img = imageio.imread(path)
    im = np.array(img,dtype='uint8')
    return im

def writeImg(path,buf):
    imageio.imwrite(path,buf)

In [211]:
def part_median_filter(local_data):
    part_id = local_data[0]
    first   = int(local_data[1])
    end     = int(local_data[2])
    buf     = np.array(local_data[3])
    nx=buf.shape[0]
    ny=buf.shape[1]

    ########################################
    #
    # CREATE NEW BUF WITH MEDIAN FILTER SOLUTION
    #
    new_buf=np.array(buf[first:end,:,:],dtype='uint8')
    
    ##########################################
    #
    # TODO COMPUTE MEDIAN FILTER
    #
    new_buf = np.array(median_filter(new_buf, size=(3,3,3)),dtype='uint8')
    
    ##########################################
    #
    # RETURN LOCAL IMAGE PART
    #
    return part_id,new_buf

In [131]:
data_dir = ''
file = os.path.join(data_dir,'lena_noisy.jpg')
img_buf=readImg(file)
print('SHAPE',img_buf.shape)
# print('IMG\n',img_buf)
nx=img_buf.shape[0]
ny=img_buf.shape[1]

SHAPE (128, 128, 3)


In [200]:
###########################################################################
#
# SPLT IMAGES IN NB_PARTITIONS PARTS
nb_partitions = 8
print("NB PARTITIONS : ",nb_partitions)
data=[]
begin=0
block_size=nx/nb_partitions
for ip in range(nb_partitions):
    end=min(begin+block_size,nx)
    data.append([ip,begin,end,img_buf])
    begin=end

NB PARTITIONS :  8


In [201]:
###########################################################################
#
# CREATE SPARKCONTEXT
# sc =SparkContext()
data_rdd = sc.parallelize(data,nb_partitions)

In [213]:
###########################################################################
#
# PARALLEL MEDIAN FILTER COMPUTATION
result_rdd = data_rdd.map(part_median_filter)
result_data = result_rdd.collect()

# new_img_buf=np.array([nx,ny],dtype='uint8')

In [281]:
result_data = np.array(result_data)
# print(result_data[:,1][:].shape)


# for i, t in enumerate(result_data[:,1][:]) :
#     # writeImg('t'+str(i)+'.jpg', t)
#     print(t.shape)
    
#     # if(i < 7)
    
writeImg('prout.jpg', np.concatenate(result_data[:,1][:]))
# print(concatenated_rdd.collect()[1][1].shape)
# writeImg('test4.png',concatenated_rdd.collect()[1][1])

In [275]:
# concatenated_rdd = result_rdd.reduce(lambda x,y: np.concatenate((x, y)))
# print(concatenated_rdd)
# res = concatenated_rdd.collect()

[0 array([[[226, 158, 137],
           [228, 163, 137],
           [228, 165, 125],
           ...,
           [178, 140, 112],
           [178, 154, 136],
           [203, 154, 152]],

          [[208, 159, 150],
           [224, 165, 157],
           [207, 165, 139],
           ...,
           [177, 132, 104],
           [156, 134, 101],
           [154, 134,  97]],

          [[197, 158, 149],
           [203, 160, 149],
           [200, 161, 147],
           ...,
           [136, 112,  91],
           [134, 112,  71],
           [134, 115,  88]],

          ...,

          [[224, 160, 129],
           [220, 169, 121],
           [220, 170, 106],
           ...,
           [ 91,  57,  54],
           [ 90,  57,  54],
           [ 89,  54,  53]],

          [[217, 160, 140],
           [220, 165, 140],
           [222, 168, 121],
           ...,
           [ 92,  57,  54],
           [ 92,  57,  54],
           [ 93,  61,  53]],

          [[208, 163, 140],
           [217, 163, 144]

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [255]:
# print(concatenated_rdd[1][0].shape)

AttributeError: 'PipelinedRDD' object has no attribute 'agg'

In [14]:
###########################################################################
#
# COMPUTE NEW IMAGE RESULTS FROM RESULT RDD
# TODO
print('CREATE NEW PICTURE FILE')
filter_file = os.path.join(data_dir,'lena_filter.jpg')
writeImg(filter_file,new_img_buf)

CREATE NEW PICTURE FILE


ValueError: Image must be 2D (grayscale, RGB, or RGBA).

# Runenv

In [18]:
data = np.array(data)
# start = data[:,1][0]
# end = data[:,2][0]
# print( start, end )
# print(np.array(data[:,3][0]).shape)
# # new_buf=np.array(buf[first:end,first:end,:],dtype='uint8')

MemoryError: 